---

In [ ]:
import mlflow
import sys
sys.path.append("../../")
from ML.utils.mlflow_flow import set_tracking


---

In [ ]:
sys.path.append("../models/ensemble_elnet_lgbm")

MODEL_NAME = 'elnet_lgbm'
set_tracking("https://dagshub.com/Yosesotomayor/retoCasas_v2.mlflow")

model_r = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@champion")


---